In [1]:
!pip install Fiona

     |████████████████████████████████| 26.5 MB 184 kB/s eta 0:00:011


In [78]:
!pip install colour

In [51]:
import fiona
import json
import pandas as pd

In [81]:
from sklearn.preprocessing import MinMaxScaler
from colour import Color

In [22]:
with fiona.open("./shp/sdei-global-uhi-2013.shp") as src:
    cities = list(src.values())

In [37]:
cities[200]['properties']

OrderedDict([('ISOURBID', 'SWE1006'),
             ('ISO3', 'SWE'),
             ('URBID', 1006),
             ('NAME', 'Arvidsjaur'),
             ('SCHNM', 'ARVIDSJAUR'),
             ('ES90POP', 5465.0),
             ('ES95POP', 5087.0),
             ('ES00POP', 4735.0),
             ('SQKM_FINAL', 154.561),
             ('URB_D_MEAN', 21.2107356322),
             ('BUF_D_MEAN', 20.9873194535),
             ('D_T_DIFF', 0.223416178703),
             ('URB_N_MEAN', 5.50245977011),
             ('BUF_N_MEAN', 5.32980481457),
             ('N_T_DIFF', 0.172654955541),
             ('LATITUDE', 65.5915918716),
             ('LONGITUDE', 19.2188733644)])

In [135]:
cities_filtered = [{'lat': city['properties']['LATITUDE'],'lng': city['properties']['LONGITUDE'], 'temp': city['properties']['N_T_DIFF'], 'name': city['properties']['NAME']} for city in cities if city['properties']['ES00POP'] > 1_000_000]

In [136]:
df = pd.DataFrame(cities_filtered)

In [137]:
df

,lat,lng,temp,name
0,60.463127,24.878620,-0.086599,Helsinki
1,59.619607,10.496465,0.216821,Oslo
2,59.863408,30.380459,0.672213,Sankt Peterburg
3,59.350440,17.969708,-0.394412,Stockholm
4,58.001796,56.012627,0.228605,Perm
...,...,...,...,...
386,-34.771678,-55.986049,0.593673,MONTEVIDEO
387,-36.881600,174.782135,0.468551,Auckland
388,-37.865565,145.082893,1.415198,Melbourne
389,50.459835,4.314236,0.716964,Charleroi


In [155]:
# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
# df_norm = pd.DataFrame(scaler.fit_transform(df_cars), columns=df_cars.columns)
df_norm = scaler.fit_transform(df[['temp']].to_numpy())
# df_norm = pd.DataFrame(df_norm, columns=df.columns)

In [156]:
df_final = df.copy()
df_final['temp_norm'] = df_norm

In [164]:
df_final['temp_rounded'] = round(df_final['temp'], 2)

In [165]:
df_final 

,lat,lng,temp,name,temp_norm,color,temp_rounded
0,60.463127,24.878620,-0.086599,Helsinki,0.367721,#5e00a1,-0.09
1,59.619607,10.496465,0.216821,Oslo,0.407551,#690096,0.22
2,59.863408,30.380459,0.672213,Sankt Peterburg,0.467332,#780087,0.67
3,59.350440,17.969708,-0.394412,Stockholm,0.327313,#5400ab,-0.39
4,58.001796,56.012627,0.228605,Perm,0.409098,#690096,0.23
...,...,...,...,...,...,...,...
386,-34.771678,-55.986049,0.593673,MONTEVIDEO,0.457022,#75008a,0.59
387,-36.881600,174.782135,0.468551,Auckland,0.440597,#70008f,0.47
388,-37.865565,145.082893,1.415198,Melbourne,0.564865,#8f0070,1.42
389,50.459835,4.314236,0.716964,Charleroi,0.473206,#780087,0.72


# Coloring

In [158]:
blue = Color("blue")
colors = list(blue.range_to(Color("red"), 101))

In [159]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [160]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1='blue' #blue
c2='red' #green
n=100

colors = [colorFader(c1,c2,x/n) for x in range(n+1)]

In [166]:
df_final['color'] = list(map(lambda x: colors[int(x)], round(df_final['temp_norm'], 2)*100))
df_final

,lat,lng,temp,name,temp_norm,color,temp_rounded
0,60.463127,24.878620,-0.086599,Helsinki,0.367721,#5e00a1,-0.09
1,59.619607,10.496465,0.216821,Oslo,0.407551,#690096,0.22
2,59.863408,30.380459,0.672213,Sankt Peterburg,0.467332,#780087,0.67
3,59.350440,17.969708,-0.394412,Stockholm,0.327313,#5400ab,-0.39
4,58.001796,56.012627,0.228605,Perm,0.409098,#690096,0.23
...,...,...,...,...,...,...,...
386,-34.771678,-55.986049,0.593673,MONTEVIDEO,0.457022,#75008a,0.59
387,-36.881600,174.782135,0.468551,Auckland,0.440597,#70008f,0.47
388,-37.865565,145.082893,1.415198,Melbourne,0.564865,#8f0070,1.42
389,50.459835,4.314236,0.716964,Charleroi,0.473206,#780087,0.72


In [167]:
df_final.to_dict('records')

[{'lat': 60.4631271879,
  'lng': 24.8786202711,
  'temp': -0.0865990166868,
  'name': 'Helsinki',
  'temp_norm': 0.36772058693269666,
  'color': '#5e00a1',
  'temp_rounded': -0.09},
 {'lat': 59.619607269,
  'lng': 10.4964654141,
  'temp': 0.216821190899,
  'name': 'Oslo',
  'temp_norm': 0.4075513140528623,
  'color': '#690096',
  'temp_rounded': 0.22},
 {'lat': 59.8634075986,
  'lng': 30.3804588179,
  'temp': 0.672213001218,
  'name': 'Sankt\xa0Peterburg',
  'temp_norm': 0.4673317323509586,
  'color': '#780087',
  'temp_rounded': 0.67},
 {'lat': 59.350440239,
  'lng': 17.9697084,
  'temp': -0.394412228741,
  'name': 'Stockholm',
  'temp_norm': 0.32731317916356983,
  'color': '#5400ab',
  'temp_rounded': -0.39},
 {'lat': 58.0017958186,
  'lng': 56.0126270496,
  'temp': 0.22860545831,
  'name': 'Perm',
  'temp_norm': 0.40909826421578405,
  'color': '#690096',
  'temp_rounded': 0.23},
 {'lat': 56.8386485807,
  'lng': 60.4845660074,
  'temp': 0.0551675457489,
  'name': 'Jekaterinburg',
  '

In [168]:
with open('cities.json', 'w') as w:
    json.dump(df_final.to_dict('records'), w)